In [1]:
# MeCabのインストール
!apt install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3
# mecab-ipadic-NEologdのインストール
# !apt install git make curl xz-utils file
# !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
# !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a
# !ln -s /etc/mecabrc /usr/local/etc/mecabrc

The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.



In [2]:
# !echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

In [3]:
import pandas as pd
import os
from glob import glob
import linecache
import random
import numpy as np
dataset = pd.read_csv('text_data.csv', encoding="utf_8_sig")

In [4]:
# dataset = pd.read_csv('text_data.csv')
dataset = dataset['title']
dataset = pd.DataFrame(dataset)
dataset.head()

,title
0,芥川賞受賞の又吉「印税のほとんどは事務所に引かれてしまうため手元に残る金額は8万円」
1,華原朋美、「PON」の月曜レギュラーを降板か！？まったく空気読めない言動連発、現場から酷評噴出
2,片岡愛之助の公演に藤原紀香からピンクの胡蝶蘭が届く！ 花言葉は「愛しています」
3,池田エライザ、自撮りのコツを伝授 「上から撮るとかわいらしく映る」
4,片岡愛之助、紀香と交際し豹変…謙虚さ失い、報道陣をバカにしてケンカ売る発言を連発


In [5]:
import MeCab
import subprocess
import re
import torch

# tagger = MeCab.Tagger("-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
tagger = MeCab.Tagger("-Owakati")
# mecabTagger = MeCab.Tagger("-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")

In [6]:
#単語ベクトル作成
def make_wakati(sentence):
    # 半角全角英数字除去
    #print(sentence)
    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', "", sentence)
    #print(sentence)
    # 記号もろもろ除去
#     sentence = re.sub(r'[\．_－―─＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
    #print(sentence)
    #スペース削除(スペースで変に区切られた単語があったため)
    sentence = sentence.replace(' ', '')
    # MeCabで分かち書き
    sentence = tagger.parse(sentence)
    #print(sentence)
    
#     tagger.parse("")
    node = tagger.parseToNode(sentence)
    keywords = []
    while node:
        if node.feature.split(",")[0] == u"名詞":
            keywords.append(node.surface)
        elif node.feature.split(",")[0] == u"形容詞":
            keywords.append(node.feature.split(",")[6])
        elif node.feature.split(",")[0] == u"動詞":
            keywords.append(node.feature.split(",")[6])
        node = node.next
#     print(keywords)
    return keywords

In [7]:
# 単語ID辞書を作成する
word2index = {}
word_count = {}
for title in dataset["title"]:
    wakati = make_wakati(title)
    for word in wakati:
        if not word in word_count:
            word_count[word] = 0
        word_count[word] += 1
        if word in word2index: continue
        word2index[word] = len(word2index)
print("vocab size : ", len(word2index))

vocab size :  4371


In [8]:
# word_count

In [9]:
word_counts = sorted(word_count.items(), key=lambda x:x[1], reverse=True)
word_counts

[('する', 269),
 ('れる', 108),
 ('いる', 98),
 ('ベッキー', 89),
 ('者', 63),
 ('不倫', 55),
 ('結婚', 54),
 ('なる', 52),
 ('の', 49),
 ('人', 48),
 ('批判', 45),
 ('視聴', 42),
 ('川谷', 40),
 ('ない', 38),
 ('こと', 37),
 ('声', 35),
 ('殺到', 34),
 ('年', 34),
 ('中', 32),
 ('率', 32),
 ('主演', 32),
 ('明かす', 31),
 ('関係', 30),
 ('ゲス', 30),
 ('騒動', 30),
 ('ファン', 28),
 ('暴露', 27),
 ('清原', 27),
 ('女性', 27),
 ('事務所', 26),
 ('ドラマ', 26),
 ('謝罪', 25),
 ('出演', 25),
 ('妻', 25),
 ('疑惑', 25),
 ('中居', 24),
 ('さん', 24),
 ('ある', 23),
 ('紅白', 23),
 ('絵', 23),
 ('すぎる', 23),
 ('逮捕', 23),
 ('ジャ', 23),
 ('ニーズ', 23),
 ('氏', 23),
 ('理由', 22),
 ('番組', 22),
 ('音', 22),
 ('発覚', 22),
 ('木村', 21),
 ('後', 21),
 ('ら', 21),
 ('メンバー', 21),
 ('紗', 21),
 ('できる', 20),
 ('られる', 20),
 ('せる', 20),
 ('交際', 19),
 ('さ', 19),
 ('アナ', 19),
 ('嵐', 19),
 ('噂', 19),
 ('告白', 19),
 ('てる', 19),
 ('報道', 19),
 ('出る', 18),
 ('語る', 18),
 ('激怒', 18),
 ('役', 18),
 ('加藤', 18),
 ('芸能', 17),
 ('狩野', 17),
 ('日', 17),
 ('熱愛', 17),
 ('見る', 16),
 ('離婚', 16),
 ('自分', 16),
 ('愛

In [10]:
# word2index

In [11]:
print("---------------------------------")
# テスト
#test = "例のあのメニューも！ニコニコ超会議のフードコートメニュー14種類紹介（前半）"
test = "この間の業務手伝っていただき、ありがとうございました！！\n本日はお疲れ様です。良い成果が出せそうです。"
#test = "【終了しました】ソニアリキエル、アユーラ、アナスイコスメティックスのチークを6名様にプレゼント"
print(make_wakati(test))

---------------------------------
['この間', '業務', '手伝う', 'いただく', '本日', '疲れ', '様', '良い', '成果', '出せる', 'そう']


---

In [21]:
import gensim
# model = gensim.models.Word2Vec.load('ja/ja.bin')
# model = gensim.models.KeyedVectors.load_word2vec_format('ja/ja.bin', binary=True)
model = gensim.models.KeyedVectors.load_word2vec_format('model.vec', binary=False)

In [20]:
model.most_similar("業務")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('サービス', 0.5841866731643677),
 ('事業', 0.5536484718322754),
 ('コンサルティング', 0.5295538902282715),
 ('コールセンター', 0.523859441280365),
 ('受託', 0.5207476615905762),
 ('アウトソーシング', 0.48645949363708496),
 ('窓口', 0.47717025876045227),
 ('管理', 0.47622430324554443),
 ('リテール', 0.46438294649124146),
 ('委託', 0.46361541748046875)]

In [22]:
# model['感謝']
model.most_similar(positive=['日本人'])

[('韓国人', 0.7338134050369263),
 ('中国人', 0.717720627784729),
 ('アメリカ人', 0.6725355386734009),
 ('日本人女性', 0.6723321676254272),
 ('外国人', 0.642046332359314),
 ('フィリピン人', 0.6264426708221436),
 ('欧米人', 0.6217868328094482),
 ('アジア人', 0.6192302703857422),
 ('台湾人', 0.6034690141677856),
 ('日系人', 0.5906497240066528)]

In [84]:
sample = ["賞賛", "応援", "労い", "尊敬", "挨拶", "同意", "自己便益", "願い", "依頼", "指示", "共感", "鼓舞", "喜び", "祈り", "詫び", "雑談"]
data = []
for type_ in sample:
    try:
        tmp = model.most_similar(positive=type_)
        data.append(tmp)
    except:
        print('word’' + type_ + '’not in vocabulary')
        A = type_
sample.remove(A)
print(sample)

word’自己便益’not in vocabulary
['賞賛', '応援', '労い', '尊敬', '挨拶', '同意', '願い', '依頼', '指示', '共感', '鼓舞', '喜び', '祈り', '詫び', '雑談']


In [90]:
# data

In [88]:
data = pd.DataFrame(data, index=sample)
data = data.T

In [89]:
data

,賞賛,応援,労い,尊敬,挨拶,同意,願い,依頼,指示,共感,鼓舞,喜び,祈り,詫び,雑談
0,"(称賛, 0.8758201599121094)","(応援団, 0.6823321580886841)","(ねぎらい, 0.617759108543396)","(敬愛, 0.7310981154441833)","(あいさつ, 0.7708175182342529)","(承諾, 0.7093747854232788)","(願っ, 0.757114052772522)","(頼ま, 0.6608020067214966)","(命令, 0.657062828540802)","(親近感, 0.6171678304672241)","(士気, 0.6211110353469849)","(喜ん, 0.7070636749267578)","(祈る, 0.7169433832168579)","(詫びる, 0.6516720652580261)","(会話, 0.5758863687515259)"
1,"(絶賛, 0.7380796670913696)","(応援歌, 0.5667240023612976)","(ねぎらう, 0.6052253246307373)","(敬意, 0.669126570224762)","(ご挨拶, 0.5933835506439209)","(了承, 0.6695442795753479)","(願う, 0.7452517747879028)","(要請, 0.6131413578987122)","(指令, 0.5473251342773438)","(感情, 0.596405029296875)","(奮い立た, 0.5846611261367798)","(喜ぶ, 0.6653314232826233)","(捧げる, 0.6746728420257568)","(謝罪, 0.6201756000518799)","(世間話, 0.5678499937057495)"
2,"(賛辞, 0.6888724565505981)","(応援団長, 0.541817307472229)","(ねぎらわ, 0.533714771270752)","(賞賛, 0.6277339458465576)","(新年の挨拶, 0.5791879892349243)","(合意, 0.6416131258010864)","(叶え, 0.708531379699707)","(引き受け, 0.599562406539917)","(命じ, 0.5420212745666504)","(同情, 0.5840601921081543)","(激励, 0.569572925567627)","(歓喜, 0.6540846824645996)","(祈っ, 0.6557257771492004)","(謝っ, 0.595147967338562)","(談義, 0.5427384972572327)"
3,"(褒め, 0.6615880727767944)","(サポーター, 0.5383031368255615)","(ねぎらっ, 0.5245795249938965)","(人柄, 0.6188326478004456)","(拍手, 0.5666038990020752)","(了解, 0.6381179094314575)","(叶える, 0.6832717657089233)","(快諾, 0.5869093537330627)","(待機命令, 0.541455864906311)","(尊敬, 0.5660059452056885)","(高揚, 0.5626145601272583)","(驚き, 0.6459325551986694)","(捧げ, 0.6397427320480347)","(激怒, 0.5917230844497681)","(内輪ネタ, 0.4905930161476135)"
4,"(尊敬, 0.6277340650558472)","(がんばろう, 0.5364265441894531)","(労う, 0.5175854563713074)","(称賛, 0.6175302863121033)","(よろしくお願いします, 0.5635450482368469)","(承認, 0.6189716458320618)","(叶う, 0.6427521705627441)","(頼ん, 0.5855815410614014)","(要請, 0.5322249531745911)","(感動, 0.5554643273353577)","(愛国心, 0.5339468717575073)","(悲しみ, 0.6435604095458984)","(祈祷, 0.636040210723877)","(非礼, 0.5887055397033691)","(飲み会, 0.4794546365737915)"
5,"(評し, 0.6254720091819763)","(全力応援, 0.5290887951850891)","(激励, 0.5153719186782837)","(慕っ, 0.6097602844238281)","(お疲れさま, 0.5571840405464172)","(拒絶, 0.601908802986145)","(想い, 0.6147809624671936)","(申し出, 0.5685433745384216)","(命じる, 0.5291622877120972)","(心情, 0.5494885444641113)","(叱咤, 0.5288562774658203)","(感動, 0.6360564231872559)","(礼拝, 0.6317447423934937)","(申し訳な, 0.5779212117195129)","(話, 0.47583115100860596)"
6,"(高い評価, 0.6185067892074585)","(声援, 0.527733325958252)","(励まし, 0.5138002038002014)","(心酔, 0.6058896780014038)","(祝辞, 0.5566996932029724)","(賛成, 0.6014900803565979)","(聞き届け, 0.6050291061401367)","(頼み, 0.5668511986732483)","(従わ, 0.5135115385055542)","(好意, 0.547883152961731)","(団結, 0.4983225464820862)","(嬉し, 0.6308411955833435)","(聖なる, 0.6033302545547485)","(懇願, 0.574499249458313)","(話し込ん, 0.47241610288619995)"
7,"(激賞, 0.613785445690155)","(六甲おろし, 0.527489423751831)","(挨拶, 0.5039975643157959)","(慕わ, 0.5991255044937134)","(最後の挨拶, 0.5536990165710449)","(拒否, 0.5996518731117249)","(願い事, 0.6008279323577881)","(舞い込む, 0.5567319989204407)","(厳命, 0.5096352696418762)","(理解, 0.5453704595565796)","(称賛, 0.49788960814476013)","(感激, 0.6277530789375305)","(お祈り, 0.6028725504875183)","(言付け, 0.5733970403671265)","(やり取り, 0.46040013432502747)"
8,"(称え, 0.6094597578048706)","(応援合戦, 0.5261664986610413)","(慰労, 0.5008108615875244)","(親交, 0.5869675278663635)","(お辞儀, 0.5478978157043457)","(申し出, 0.5869439840316772)","(懇願, 0.5893115997314453)","(打診, 0.5528178215026855)","(助言, 0.5088523030281067)","(シンパシー, 0.544873058795929)","(励まし, 0.49689173698425293)","(感謝, 0.6260571479797363)","(ささげる, 0.5980609655380249)","(諭し, 0.5667544603347778)","(井戸端会議, 0.45968782901763916)"
9,"(評さ, 0.609194278717041)","(応援ソング, 0.525647759437561)","(感謝, 0.49625539779663086)","(憧れ, 0.5833450555801392)","(ごきげんよう, 0.5402970910072327)","(提案, 0.5829447507858276)","(ほしい, 0.5862970352172852)","(請ける, 0.54940319